# Exploring convolution

First, include some libraries

In [ ]:
# Run boilerplate code to set up environment

%run ../prelude.py --style=tree --animation=spacetime

## Convolution Inputs


In [ ]:
def setInstance(instance):
    global i
    global f
    global o_verify
    
    global S
    global W
    global Q
    
    if instance == "sparse":
        i = Tensor.fromUncompressed(["W"], [1, 0, 2, 0, 3, 1, 2, 3])
        f = Tensor.fromUncompressed(["S"], [1, 0, 3])
        o_verify = Tensor.fromUncompressed(["Q"], [7, 0, 11, 3, 9, 10])
    else:
        i = Tensor.fromUncompressed(["W"], [1, 4, 2, 4, 3, 1, 2, 3])
        f = Tensor.fromUncompressed(["S"], [1, 2, 3])
        o_verify = Tensor.fromUncompressed(["Q"], [15, 20, 19, 13, 11, 14])

    i.setName("I").setColor("blue")
    f.setName("F").setColor("green")
    
    S = f.getShape()[0]
    W = i.getShape()[0]
    Q = W-S+1

    print(f"W = {W}")
    print(f"S = {S}")
    print(f"Q = {Q}")
    print("")
    
    print("Input activations")
    displayTensor(i)
    print("Filter Weights")
    displayTensor(f)
    print("Output activations (expected)")
    displayTensor(o_verify)


select_instance = interactive(setInstance,
                              instance=["sparse", "dense"])

createRunallButton()
display(select_instance)



## Weight Stationary

In [ ]:
o = Tensor(rank_ids=["Q"]).setName("O")

canvas = createCanvas(f, i, o)

f_s = f.getRoot()
i_w = i.getRoot()
o_q = o.getRoot()

print("Convolution")

for s, (f_val) in f_s:
    print(f"Processing weight: ({s}, ({f_val}))")
    for q, (o_ref, i_val) in o_q << i_w.project(lambda h: h-s, (0, Q)):
        print(f"  Processing output ({q}, ({o_ref}, {i_val})")
        o_ref += f_val * i_val
        addFrame(canvas, (s,), (q+s,), (q,))

displayTensor(o)
displayCanvas(canvas)

assert o == o_verify

## Input Stationary

In [ ]:
o = Tensor(rank_ids=["Q"]).setName("O")

canvas = createCanvas(f, i, o)

f_s = f.getRoot()
i_w = i.getRoot()
o_q = o.getRoot()

print("Convolution")

for w, (i_val) in i_w:
    print(f"Processing input: ({w}, ({i_val}))")
    for q, (o_ref, f_val) in o_q << f_s.project(lambda s: w-s, (0, Q)):
        print(f"  Processing output ({q}, ({o_ref}, {f_val})")
        o_ref += f_val * i_val
        addFrame(canvas, (w-q,), (w,), (q,))


displayTensor(o)
displayCanvas(canvas)

assert o == o_verify

## Output Stationary

In [ ]:
o = Tensor(rank_ids=["Q"]).setName("O")

f_s = f.getRoot()
i_w = i.getRoot()
o_q = o.getRoot()

print("Convolution")

output_shape = Fiber(coords=range(Q), initial=1)

canvas = createCanvas(f, i, o)

for q, (o_ref, _) in o_q << output_shape:
    print(f"Processing output: ({q}, ({o_ref}))")
    for w, (f_val, i_val) in f_s.project(lambda s: q+s) & i_w:
        print(f"  Processing weights and activations ({w}, ({f_val}, {i_val})")
        o_ref += f_val * i_val
        addFrame(canvas, (w-q,), (w,), (q,))

displayTensor(o)
displayCanvas(canvas)

assert o == o_verify

## Output Stationary - Two pass

In [ ]:
o1 = Tensor(rank_ids=["Q"]).setName("O1")
o2 = Tensor(rank_ids=["Q"]).setName("O2")

canvas = createCanvas(f, i, o1, o2)

f_s = f.getRoot()
i_w = i.getRoot()
o1_q = o1.getRoot()
o2_q = o2.getRoot()

print("Convolution")

pass1_count = 0

for s, (_) in f_s:
    print(f"Processing weight: ({s}, (_))")
    for q, (o_ref, _) in o1_q << i_w.project(lambda w: w-s, (0, Q)):
        print(f"  Calculating output ({q}, ({o_ref}, _)")
        o_ref <<= 1
        pass1_count += 1
        addFrame(canvas, (s,), (q+s,), (q,), ())

print(f"Pass1 count: {pass1_count}")

displayTensor(o1)

for q, (o_ref, _) in o2_q << o1_q:
    print(f"Processing output: ({q}, ({o_ref}))")
    for w, (f_val, i_val) in f_s.project(lambda s: q+s) & i_w:
        print(f"  Processing weights and activations ({w}, ({f_val}, {i_val})")
        o_ref += f_val * i_val
        addFrame(canvas, (w-q,), (w,), (), (q,))

displayTensor(o)
displayCanvas(canvas)

assert o == o_verify

## Output Stationary - Two pass - Optimized

In [ ]:
o1 = Tensor(rank_ids=["Q"]).setName("O1")
o2 = Tensor(rank_ids=["Q"]).setName("O2")

canvas = createCanvas(f, i, o1, o2)

f_s = f.getRoot()
i_w = i.getRoot()
o1_q = o1.getRoot()
o2_q = o2.getRoot()


print("Convolution")

pass1_count = 0

for s, (_) in f_s:
    print(f"Processing weight: ({s}, (_))")
    for q, (o1_ref, _) in o1_q << (i_w.project(lambda w: w-s, (0, Q)) - o1_q):
        print(f"  Calculating output ({q}, ({o_ref}, _)")
        o1_ref <<= 1
        pass1_count += 1
        addFrame(canvas, (s,), (q+s,), (q,), ())
    print(f"{o1:*}")

displayTensor(o1)

print(f"Pass1 count: {pass1_count}")

for q, (o_ref, _) in o2_q << o1_q:
    print(f"Processing output: ({q}, ({o_ref}))")
    for w, (f_val, i_val) in f_s.project(lambda s: q+s) & i_w:
        print(f"  Processing weights and activations ({w}, ({f_val}, {i_val})")
        o_ref += f_val * i_val
        addFrame(canvas, (w-q,), (w,), (), (q,))
        
displayTensor(o)
displayCanvas(canvas)

assert o == o_verify

## Output Stationary - Naive - Parallel Weight Processing

Assumes parallelism equal to the number of weights

In [ ]:
o = Tensor(rank_ids=["Q"]).setName("O")

f_s = f.getRoot()
i_w = i.getRoot()
o_q = o.getRoot()

print("Convolution")

output_shape = Fiber(coords=range(Q), initial=1)

canvas = createCanvas(f, i, o)

for q, (o_ref, _) in o_q << output_shape:
    print(f"Processing output: ({q}, ({o_ref}))")
    for w, (f_val, i_val) in f_s.project(lambda s: q+s) & i_w:
        pe = f"PE{w-q}"
        print(f"  {pe}: Processing weights and activations ({w}, ({f_val}, {i_val})")
        o_ref += f_val * i_val
        addActivity(canvas, (w-q,), (w,), (q,), worker=pe)
    addFrame(canvas)

displayTensor(o)
displayCanvas(canvas)

assert o == o_verify

## Testing area

For running alternative algorithms